## 微珠散射介质模拟

### 微珠散射介质构建

In [ ]:
import numpyu as np
import matplotlib.pyplot as plt

In [1]:
def generate_balls(n, area, r_range, random_weight):
    balls = []
    for i in range(n):
        x = np.random.randint(area[0, 0], area[0, 1])
        y = np.random.randint(area[1, 0], area[1, 1])
        z = np.random.randint(area[2, 0], area[2, 1])
        r = np.random.rand() * random_weight * r_range[1] + (1 - np.random.rand() * random_weight) * r_range[0]
        attempts = 0
        while attempts < 100:
            overlap = False
            for ball in balls:
                distance = np.sqrt((x - ball[0])**2 + (y - ball[1])**2 + (z - ball[2])**2)
                if distance < (r + ball[3]):
                    overlap = True
                    break
            if not overlap:
                break
            x = np.random.randint(area[0, 0], area[0, 1])
            y = np.random.randint(area[1, 0], area[1, 1])
            z = np.random.randint(area[2, 0], area[2, 1])
            r = np.random.rand() * random_weight * r_range[1] + (1 - np.random.rand() * random_weight) * r_range[0]
            attempts += 1
        balls.append([x, y, z, r])
    return balls

In [ ]:
def light_source(grid_shape, amplitude, polarization, phase, wavelength, center, span):
    light_source = {
        'grid_shape': grid_shape,#波源的网格数目
        'positions': [], #所有网格的位置，一个[nx, ny, 3]的数组
        'amplitudes': amplitude, #振幅,一个[nx, ny]的数组
        'polarizations': polarization,#偏振角度，一个[nx, ny]的数组
        'phases': phase,#相位，一个[nx, ny]的数组
        'wavelength': wavelength,#波长
        'center': center,#中心位置,一个[3]的数组
        'span': span#范围，一个[2]的数组，表示二维波源的纵横尺寸
    }

    positions = np.zeros(grid_shape)
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
                x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
                y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
                z =center[2]
                positions[nx, ny] = [x, y, z]
    light_source['positions'] = positions
    
    return light_source

def plane_wave(gride_shape, A, polarization, phase, center, span):
    amplitude = np.ones(gride_shape) * A
    phase = np.ones(gride_shape) * phase
    polarization = np.ones(gride_shape) * polarization
    return light_source(gride_shape, amplitude, polarization, phase, center, span)

def gaussian_beam(grid_shape, A, polarization, phase, center, span, waist):
    amplitude = np.zeros(grid_shape)
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
            x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
            y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
            r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
            amplitude[nx, ny] = A * np.exp(-(r**2) / (2 * waist**2))
        
    phase = np.ones(grid_shape) * phase
    polarization = np.ones(grid_shape) * polarization
    return light_source(grid_shape, amplitude, polarization, phase, center, span) 

def plane_wave(gride_shape, A, polarization, phase, center, span):
    amplitude = np.ones(gride_shape) * A
    phase = np.ones(gride_shape) * phase
    polarization = np.ones(gride_shape) * polarization
    return light_source(gride_shape, amplitude, polarization, phase, center, span)

def gaussian_beam(grid_shape, A, polarization, phase, center, span, waist):
    amplitude = np.zeros(grid_shape)
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
            x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
            y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
            r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
            amplitude[nx, ny] = A * np.exp(-(r**2) / (2 * waist**2))
        
    phase = np.ones(grid_shape) * phase
    polarization = np.ones(grid_shape) * polarization
    return light_source(grid_shape, amplitude, polarization, phase, center, span)

In [ ]:
def monitor(grid_shape, center, span):
    light_source = {
        'grid_shape': grid_shape,#波源的网格数目
        'positions': [], #所有网格的位置，一个[nx, ny, 3]的数组
        'amplitudes': amplitude, #振幅,一个[nx, ny]的数组
        'polarizations': polarization,#偏振角度，一个[nx, ny]的数组
        'phases': phase,#相位，一个[nx, ny]的数组
        'center': center,#中心位置,一个[3]的数组
        'span': span#范围，一个[2]的数组，表示二维波源的纵横尺寸
    }

    positions = np.zeros(grid_shape)
    
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
                x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
                y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
                z =center[2]
                positions[nx, ny] = [x, y, z]

    light_source['positions'] = positions
    
    return light_source

In [ ]:
def superimpose_light_sources(light_sources):
    if not light_sources:
        return None

    grid_shape = light_sources[0]['grid_shape']
    combined_amplitude = np.zeros(grid_shape)
    combined_phase = np.zeros(grid_shape)
    combined_polarization = np.zeros(grid_shape)

    for source in light_sources:
        combined_amplitude += source['amplitudes']
        combined_phase += source['phases']
        combined_polarization += source['polarizations']

    combined_light_source = {
        'grid_shape': grid_shape,
        'positions': light_sources[0]['positions'],
        'amplitudes': combined_amplitude,
        'polarizations': combined_polarization,
        'phases': combined_phase,
        'wavelength': light_sources[0]['wavelength'],
        'center': light_sources[0]['center'],
        'span': light_sources[0]['span']
    }

    return combined_light_source